In [ ]:
#==============================================================================
#
# IMPORT LIBRARIES
# 
#==============================================================================
import time
import pandas as pd
#pd.options.display.max_columns = None # Shows all columns
import string as str
import numpy as np
import matplotlib.pyplot as plt
import dill
import dask
import dask.dataframe as dd
import re
from pprint import pprint

#import csv
#from googletrans import Translator
#import copy
#import numpy as np

In [ ]:
#==============================================================================
# 
# DUMP/LOAD SESSIONS
# 
#==============================================================================
# ---- FULL SESSIONS ----
# Dump
#dill.dump_session('./working.db')

# Load
#dill.load_session('working.db')

# ---- OBJECTS ----
# Dump
#dill.dump(df_raw, open("df_raw.pkl", "wb"))

# Load
#dill_file = open("df_raw.pkl", "rb")
#df_raw = dill.load(dill_file)
#dill_file.close()

In [ ]:
#==============================================================================
# 
# READ/EXTRACT RAW DATA
# 
#==============================================================================

file_train = 'data/dataset/yes_complete/train.txt'
print(pd.read_table(file_train, nrows=5, header=None))

In [ ]:
with open(file_train) as f:
    train_raw = f.readlines()

In [ ]:
playlists_train_raw = train_raw[2:]

In [ ]:
# How many unique playlists?
print('There are ', len(playlists_train_raw), ' playlists.', sep='')

In [ ]:
# Split playlist IDs and convert to integers
playlists_train_list = []
for i in range(0, len(playlists_train_raw)):
    playlists_train_list.append(list(map(int, playlists_train_raw[i].split())))

In [ ]:
# Flatmap the playlists list
df_playlists_train = []
for i in range(0, len(playlists_train_list)):
    for j in range(0, len(playlists_train_list[i])):
        df_playlists_train.append([i, playlists_train_list[i][j]])

In [ ]:
df_playlists_train = pd.DataFrame(df_playlists_train, columns=['playlist', 'song_id'])

In [ ]:
len(df_playlists_train)

In [ ]:
# Create index primary key
df_playlists_train.insert(loc=0, column='index', value=df_playlists_train.index)

In [ ]:
# ---- Merge songs on song_id ----
df_song_hash = pd.read_table('data/dataset/yes_complete/song_hash.txt', header=None, names=['song_id','song','band'])

In [ ]:
df_playlists_train = df_playlists_train.merge(df_song_hash, on='song_id')
df_playlists_train.sort_values('index', inplace=True)
df_playlists_train.reset_index(drop=True, inplace=True)

In [ ]:
df_playlists_train.head()

In [ ]:
#==============================================================================
#
# EDA
# 
#==============================================================================

In [ ]:
#song_count = pd.DataFrame(df_playlists_train.groupby(['song', 'band'])['playlist'].count().sort_values(ascending=False))
song_count = df_playlists_train.groupby(['song_id','song','band']).count().reset_index().drop(columns='index')
song_count.columns = ['song_id','song','band','count']
song_count = song_count.sort_values('count', ascending=False)
song_count.reset_index(inplace=True)
song_count = song_count[['song_id','song','band','count']]

In [ ]:
plt.figure(figsize=(10,4))
song_count['count'].hist(bins=70)
plt.xlabel('Song count')
plt.ylabel('Quantity')
plt.title('')

In [ ]:
plt.figure(figsize=(10,4))
song_count['count'].hist(bins=10000)
plt.xlabel('Song count')
plt.ylabel('Quantity')
plt.title('')
plt.xlim(0,25)

In [ ]:
plt.figure(figsize=(10,4))
song_count[song_count['count'] > 5]['count'].hist(bins=5000)
plt.xlabel('Song count')
plt.ylabel('Quantity')
plt.title('')
plt.xlim(5,20)

In [ ]:
band_count = pd.DataFrame(df_playlists_train.groupby('band')['playlist'].count().sort_values(ascending=False))
band_count.reset_index(inplace=True)
band_count.columns = ['band','count']

In [ ]:
band_count.head()

In [ ]:
plt.figure(figsize=(10,4))
band_count['count'].hist(bins=70)
plt.xlabel('Band count')
plt.ylabel('Quantity')
plt.title('')

In [ ]:
plt.figure(figsize=(10,4))
band_count['count'].hist(bins=1000)
plt.xlabel('Band count')
plt.ylabel('Quantity')
plt.title('')
plt.xlim(0,100)

In [ ]:
#==============================================================================
#
# CLEAN DATA
# 
#==============================================================================

In [ ]:
df_playlists_train.head()

In [ ]:
len(df_playlists_train)

In [ ]:
df_playlists_train = pd.merge(df_playlists_train, song_count)
df_playlists_train.sort_values(by='index', inplace=True)
df_playlists_train.reset_index(drop=True, inplace=True)

In [ ]:
# ---- PARE DATA ----
# Remove songs that show up less than 6 times
df_playlists_train = df_playlists_train[df_playlists_train['count']>=6]

In [ ]:
df_playlists_train.head()

In [ ]:
df_playlists_train.sort_values('count').head()

In [ ]:
df_playlists_train.sort_values('count').tail()

In [ ]:
# ---- REMOVE NULLS ----

In [ ]:
df_playlists_train.isnull().sum().sum()

In [ ]:
df_playlists_train.isna().sum().sum()

In [ ]:
# ---- REMOVE "", " ", "-" ----
df_playlists_train[df_playlists_train['song'] == '-'].head()

In [ ]:
df_playlists_train.drop(df_playlists_train[df_playlists_train['song']=='-'].index, inplace=True)

In [ ]:
df_playlists_train[df_playlists_train['song'] == '-']

In [ ]:
df_playlists_train[df_playlists_train['band'] == '-'].head()

In [ ]:
df_playlists_train.drop(df_playlists_train[df_playlists_train['band']=='-'].index, inplace=True)

In [ ]:
df_playlists_train[df_playlists_train['band']=='-']

In [ ]:
df_playlists_train[df_playlists_train['song']=='']

In [ ]:
df_playlists_train[df_playlists_train['band']=='']

In [ ]:
df_playlists_train[df_playlists_train['song']==' ']

In [ ]:
df_playlists_train[df_playlists_train['band']==' ']

In [ ]:
# ---- REMOVE ESCAPE CHARACTER '\' ----
df_playlists_train['song'] = [x.replace("\\","") for x in df_playlists_train['song']]
df_playlists_train['band'] = [x.replace("\\","") for x in df_playlists_train['band']]

In [ ]:
# REPLACE [] WITH ()
df_playlists_train['song'] = [x.replace("[","(") for x in df_playlists_train['song']]
df_playlists_train['song'] = [x.replace("]",")") for x in df_playlists_train['song']]
df_playlists_train['band'] = [x.replace("[","(") for x in df_playlists_train['band']]
df_playlists_train['band'] = [x.replace("]",")") for x in df_playlists_train['band']]

In [ ]:
# LOWER SONG AND BAND
df_playlists_train.head()

In [ ]:
df_playlists_train['song'] = [x.lower() for x in df_playlists_train['song']]
df_playlists_train['band'] = [x.lower() for x in df_playlists_train['band']]

In [ ]:
df_playlists_train.head()

In [ ]:
# ---- CONCAT SONG AND BAND ----
df_playlists_train['song_band'] = df_playlists_train['song'] + " [" + df_playlists_train['band'] + "]"

# Rearrange columns
df_playlists_train = df_playlists_train[['index','playlist','song_id','song','band','song_band','count']]

In [ ]:
df_playlists_train.tail()

In [ ]:
df_playlists_train['song_id'].nunique()

In [ ]:
df_playlists_train['song_band'].nunique()

In [ ]:
# Why the difference?
df_playlists_train['song_id'].nunique() - df_playlists_train['song_band'].nunique()

In [ ]:
#==============================================================================
#
# BUILD RECOMMENDER
#
#==============================================================================

In [ ]:
# ---- CREATE PLAYLIST-SONG MATRIX ----
start_clock = time.clock()

# Create song_band dummies
#playlist_song_mat_train = pd.get_dummies(df_playlists_train['song_band'], sparse=True)
playlist_song_mat_train = pd.get_dummies(df_playlists_train['song_band'])

# Insert index and playlist columns
playlist_song_mat_train.insert(loc=0, column='playlist', value=df_playlists_train['playlist'])
playlist_song_mat_train.insert(loc=0, column='index', value=df_playlists_train['index'])

end_clock = time.clock()
    
print('\n')
print('Runtime: ', round((end_clock - start_clock)/60, 2), ' min', sep='')

In [ ]:
playlist_song_mat_train.info()

In [ ]:
# CONVERT TO DASK DATAFRAME 

In [ ]:
start_clock = time.clock()
dd_playlist_song_mat_train = dd.from_pandas(data=playlist_song_mat_train, npartitions=1000)
end_clock = time.clock()

print('Runtime: ', round((end_clock - start_clock)/60, 2), ' min')

In [ ]:
dd_playlist_song_mat_train.info()

In [ ]:
# CLEAR MEMORY 

In [ ]:
%who DataFrame

In [ ]:
del(band_count, df_song_hash, playlist_song_mat_train, 
    playlists_train_list, playlists_train_raw, song_count, train_raw)

In [ ]:
# GROUP BY PLAYLIST 

In [ ]:
start_clock = time.clock()
playlist_song_mat_train = dd_playlist_song_mat_train.groupby('playlist').sum().compute()
playlist_song_mat_train.drop(columns='index', inplace=True)
end_clock = time.clock()

print('Runtime: ', round((end_clock - start_clock)/60, 2), ' min')

In [ ]:
playlist_song_mat_train.info()

In [ ]:
del(dd_playlist_song_mat_train)

In [ ]:
playlist_song_mat_train.head()

In [ ]:
# ---- CHECKS ----

In [ ]:
# Check number of Nelly's #1 songs
len(df_playlists_train[df_playlists_train['song_band']=='#1 [nelly]'])

In [ ]:
playlist_song_mat_train['#1 [nelly]'].sum()

In [ ]:
# Check length of (arbitrarily chosen) playlist 472 (partially checks for re-indexing)
sum(playlist_song_mat_train[playlist_song_mat_train.columns[playlist_song_mat_train.iloc[472,:] > 0]].iloc[472])

In [ ]:
len(df_playlists_train[df_playlists_train['playlist']==472])

In [ ]:
# Check number of playlists
df_playlists_train['playlist'].nunique()

In [ ]:
len(playlist_song_mat_train)

In [ ]:
# Check number of unique songs
df_playlists_train['song_band'].nunique()

In [ ]:
playlist_song_mat_train.shape[1]

In [ ]:
# Convert every non-zero count to just 1, ie each playlist should count each unique song as appearing only once
# regardless of how often it actually appears.
playlist_song_mat_train = playlist_song_mat_train.astype(bool).astype(int)

In [ ]:
playlist_song_mat_train['#1 [nelly]'].sum()

In [5]:
import dill
import pandas as pd

# Save out dataframes
# Dump
#dill.dump(df_playlists_train, open("df_playlists_train.pkl", "wb"))
#dill.dump(playlist_song_mat_train, open("playlist_song_mat_train.pkl", "wb"))

# Load
df_playlists_train = dill.load(open("df_playlists_train.pkl", "rb"))
playlist_song_mat_train = dill.load(open("playlist_song_mat_train.pkl", "rb"))

In [40]:
# ---- SONG OR BAND INQUIRY ----
song_band_inquiry = input('What song or band do you like? ').lower()

What song or band do you like? NeLLy


In [41]:
options = pd.DataFrame([col for col in playlist_song_mat_train.columns if song_band_inquiry in col])[0]

In [42]:
options

0                                            #1 [nelly]
1     air force ones (w/ fyjuan, ali & murphy lee) [...
2                         bajo otra luz [nelly furtado]
3                body on me (w/ akon & ashanti) [nelly]
4                    dilemma (w/ kelly rowland) [nelly]
5                                 do it [nelly furtado]
6                                          e.i. [nelly]
7                fotografia (w/ nelly furtado) [juanes]
8     give it to me (w/ nelly furtado & justin timbe...
9                       gone (w/ kelly rowland) [nelly]
10        grillz (w/ paul wall, big gipp & ali) [nelly]
11       here i am (w/ nelly & avery storm) [rick ross]
12                                 hot in herre [nelly]
13              hot-n-fun (w/ nelly furtado) [n.e.r.d.]
14                      i'm like a bird [nelly furtado]
15                   jump (w/ nelly furtado) [flo-rida]
16                                 just a dream [nelly]
17           long gone (w/ plies & chris brown) 

In [55]:
while True:
    print('\n')
    options
    try:
        song_band_number = int(input('What number? '))
    except:
        print('\n')
        print('*** Must input an integer. ***')
        continue
    else: 
        if (song_band_number in range(0,len(options))):
            song_band_choice = options[song_band_number]
            break
        else:
            print('\n')
            print('*** Choose a number from the table. ***')
            continue



What number? 3


In [50]:
song_band_choice

'#1 [nelly]'

In [51]:
'''# Return options
print('Copy the song [band] you are interested in:')
print('\n')
#pprint([col for col in playlist_song_mat_train.columns.str.lower() if song_band_inquiry.lower() in col])
pprint([col for col in playlist_song_mat_train.columns if song_band_inquiry in col])'''

"# Return options\nprint('Copy the song [band] you are interested in:')\nprint('\n')\n#pprint([col for col in playlist_song_mat_train.columns.str.lower() if song_band_inquiry.lower() in col])\npprint([col for col in playlist_song_mat_train.columns if song_band_inquiry in col])"

In [52]:
'''song_band = input('Input the song [band] of interest: ')'''

"song_band = input('Input the song [band] of interest: ')"

In [53]:
print('Top 3 suggested songs: ')
for x_song_band, x_count in \
    playlist_song_mat_train[playlist_song_mat_train[song_band_choice]==1]\
        .sum().sort_values(ascending=False)[1:4,].iteritems():
    print(' * ', x_song_band)

Top 3 suggested songs: 
 *  down on me (w/ 50 cent) [jeremih]
 *  bottoms up (w/ nicki minaj) [trey songz]
 *  how do u want it (w/ k-ci & jojo) [2pac]


In [54]:
playlist_song_mat_train[playlist_song_mat_train[song_band_choice]==1]\
    .sum().sort_values(ascending=False)

#1 [nelly]                                                12
down on me (w/ 50 cent) [jeremih]                          7
bottoms up (w/ nicki minaj) [trey songz]                   7
how do u want it (w/ k-ci & jojo) [2pac]                   7
slow jamz (w/ kanye west & jamie foxx) [twista]            7
racks [yc]                                                 6
all of the lights [kanye west]                             6
deuces [chris brown & tyga]                                6
nann nigga (w/ trina) [trick daddy]                        6
did it on' em [nicki minaj]                                6
cupid [lloyd]                                              6
walking [mary mary]                                        5
far away [marsha ambrosius]                                5
a womans worth [alicia keys]                               5
lay it down [lloyd]                                        5
like a g6 (w/ the cataracs & dec) [far east movement]      5
motivation (w/ lil wayne